Objetivo do projeto: Iremos tentar prever o preço de venda a partir das outras variáveis para cada modelo, de forma a que uma revendedora de carros, por exemplo, possa saber o qual preço pode esperar por cada carro que tiver a oportunidade de comprar.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
ford = pd.read_csv('Grupo_E_O_ford.csv')

## Exploração inicial

In [3]:
#Verificar o topo da tabela
ford.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Fiesta,2017,12000,Automatic,15944,Petrol,150,57.7,1.0
1,Focus,2018,14000,Manual,9083,Petrol,150,57.7,1.0
2,Focus,2017,13000,Manual,12456,Petrol,150,57.7,1.0
3,Fiesta,2019,17500,Manual,10460,Petrol,145,40.3,1.5
4,Fiesta,2019,16500,Automatic,1482,Petrol,145,48.7,1.0


In [4]:
#Verificar colunas e número de entradas
ford.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17965 entries, 0 to 17964
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         17965 non-null  object 
 1   year          17965 non-null  int64  
 2   price         17965 non-null  int64  
 3   transmission  17965 non-null  object 
 4   mileage       17965 non-null  int64  
 5   fuelType      17965 non-null  object 
 6   tax           17965 non-null  int64  
 7   mpg           17965 non-null  float64
 8   engineSize    17965 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 1.2+ MB


Já temos informação suficiente para gerar a tabela descritiva das variáveis exigida no relatório:

| Variável     | Escala                | Descrição                                                                                 |
|--------------|-----------------------|-------------------------------------------------------------------------------------------|
| model        | Qualitativa           | O modelo do carro.                                                                        |
| year         | Quantitativa Discreta | O ano de fabricação do carro.                                                             |
| price        | Quantitativa Contínua | O preço pelo qual o carro está sendo vendido.                                             |
| transmission | Qualitativa           | Tipo de trasmissão do carro (manual ou automática).                                       |
| mileage      | Quantitativa Contínua | Milhagem do carro (quantidade de milhas percorridas).                                     |
| fuelType     | Qualitativa           | Tipo de combustível                                                                       |
| tax          | Quantitativa Contínua | Quantia paga em imposto.                                                                  |
| mpg          | Quantitativa Contínua | Milhas percorridas por galão de combustível. Representa a eficiência energética do carro. |
| engineSize   | Quantitativa Contínua | Tamanho do Motor.                                                                         |

## Busca por inconsistência nos dados

Já pela análise acima é possível ver que não existem valores explicitamente marcados como faltantes(que estariam convertidos num objeto pd.nan).

Para continuar a busca por dados inconsistentes, vamos primeiro gerar uma tabela de frequências para todas as colunas.

In [5]:
#Gerar uma tabela de frequências pra cada coluna
for col in ford.columns:
    print(col)
    print(ford[col].value_counts(dropna=False))
    print('-'*20)
    

model
 Fiesta                   6557
 Focus                    4588
 Kuga                     2225
 EcoSport                 1143
 C-MAX                     543
 Ka+                       531
 Mondeo                    526
 B-MAX                     355
 S-MAX                     296
 Grand C-MAX               247
 Galaxy                    228
 Edge                      208
 KA                        199
 Puma                       80
 Tourneo Custom             69
 Grand Tourneo Connect      59
 Mustang                    57
 Tourneo Connect            33
 Fusion                     16
 Streetka                    2
 Ranger                      1
 Escort                      1
 Transit Tourneo             1
Name: model, dtype: int64
--------------------
year
2017    4888
2018    4014
2019    3194
2016    2331
2015    1367
2014     805
2013     609
2020     258
2012     115
2011      94
2009      91
2010      67
2008      57
2007      32
2005      16
2006      13
2004       4
2002    

Percebemos que a coluna engineSize e coluna tax possuem valores faltantes, que estão marcados como 0.

Primeiro vamos analisar as entradas com engineSize = 0.0

In [6]:
ford[ford['engineSize']==0.0].head(10)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
13862,Kuga,2019,18990,Manual,8389,Petrol,150,35.3,0.0
14064,Focus,2016,15500,Manual,28344,Diesel,20,67.3,0.0
14082,Focus,2015,14300,Manual,40457,Diesel,20,67.3,0.0
15343,KA,2018,10000,Manual,10051,Petrol,150,43.5,0.0
15500,EcoSport,2015,8500,Manual,33403,Diesel,30,61.4,0.0
15775,Focus,2017,13000,Manual,24484,Diesel,145,70.6,0.0
16312,Focus,2015,9170,Manual,28000,Petrol,20,61.4,0.0
16313,Focus,2019,18699,Manual,10000,Petrol,145,46.3,0.0
16321,Fiesta,2017,9899,Manual,10000,Petrol,0,65.7,0.0
16322,Ka+,2018,9999,Manual,15000,Petrol,145,43.5,0.0


De fato se tratam de carros com a informação faltando. Como são apenas cerca de 50 colunas de 17000, podemos simplesmente apagá-las.

In [7]:
ford = ford[ford['engineSize']!=0.0]

Em seguida, analisaremos as entradas com tax = 0

In [8]:
ford[ford['tax']==0].head(10)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
12,Fiesta,2017,11000,Manual,20978,Petrol,0,65.7,1.0
24,Fiesta,2014,6795,Manual,35299,Petrol,0,65.7,1.0
27,Fiesta,2015,8299,Manual,25935,Petrol,0,65.7,1.0
30,Fiesta,2016,7999,Manual,60730,Diesel,0,78.5,1.5
33,Fiesta,2013,7236,Manual,39090,Petrol,0,65.7,1.0
50,Fiesta,2016,6000,Manual,47613,Petrol,0,65.7,1.0
65,Fiesta,2013,5998,Manual,44304,Diesel,0,76.4,1.5
70,Fiesta,2013,7490,Manual,25000,Petrol,0,65.7,1.0
73,Fiesta,2014,6360,Manual,48653,Petrol,0,65.7,1.0
74,Fiesta,2014,6270,Manual,60888,Petrol,0,65.7,1.0


Podem ser simplesmente carros isentos de imposto. Seria necessária mais informação para definir se esses valores são inapropriados ou não.

In [17]:
#Define listas com as colunas qualitativas e quantitativas
colunas_quantitativas = list(ford.select_dtypes(include=[np.number]).columns.values)
colunas_qualitativas = list(ford.select_dtypes(exclude=[np.number]).columns.values)

print(f'Colunas quantitativas: {colunas_quantitativas}\nColunas qualitativas: {colunas_qualitativas}')

Colunas quantitativas: ['year', 'price', 'mileage', 'tax', 'mpg', 'engineSize']
Colunas qualitativas: ['model', 'transmission', 'fuelType']


In [37]:
def inspecionar_outliers(coluna):
    array_zscore = np.abs(stats.zscore(coluna))
    boolarray_zscore = array_zscore.apply(lambda x: True if x >= 3 else False)
    print(f'''Coluna: {coluna.name}
    Número de entradas: {len(coluna)}
    Número de outliers: {boolarray_zscore.sum()}
    Proporção de outliers: {boolarray_zscore.sum()/len(coluna)}
    Lista de outliers: {sorted(list(zip(list(array_zscore[boolarray_zscore]), list(coluna[boolarray_zscore]))),reverse=True)}
    \n {'-'*10} \n''')

In [38]:
for col in colunas_quantitativas:
    inspecionar_outliers(ford[col])

Coluna: year
    Número de entradas: 17914
    Número de outliers: 290
    Proporção de outliers: 0.016188455956235345
    Lista de outliers: [(21.0264652486042, 2060), (10.171011572298363, 1996), (9.196090421645158, 1998), (8.221169270991952, 2000), (7.246248120338747, 2002), (7.246248120338747, 2002), (7.246248120338747, 2002), (6.7587875450121455, 2003), (6.7587875450121455, 2003), (6.7587875450121455, 2003), (6.271326969685543, 2004), (6.271326969685543, 2004), (6.271326969685543, 2004), (6.271326969685543, 2004), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.78386639435894, 2005), (5.296405819032337, 2006), (5.296405819032337, 2006), (5.296